In [1]:
!jupyter kernelspec list

Available kernels:
  python3    C:\ProgramData\Anaconda3\envs\tf\share\jupyter\kernels\python3


In [2]:
import sys
sys.executable

'C:\\ProgramData\\Anaconda3\\envs\\tf\\python.exe'

In [3]:
sys.path

['',
 'C:\\Users\\rafae\\Downloads\\facenet\\src',
 'C:\\ProgramData\\Anaconda3\\envs\\tf\\python36.zip',
 'C:\\ProgramData\\Anaconda3\\envs\\tf\\DLLs',
 'C:\\ProgramData\\Anaconda3\\envs\\tf\\lib',
 'C:\\ProgramData\\Anaconda3\\envs\\tf',
 'C:\\ProgramData\\Anaconda3\\envs\\tf\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\envs\\tf\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\rafae\\.ipython']

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils import to_categorical

C:\Users\rafae\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
# dimensions of our images.
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\train'
validation_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\validation'
nb_train_samples = 16000
nb_validation_samples = 1600
epochs = 100
batch_size = 16

In [11]:
def save_bottlebeck_features():
    print('1')
    datagen = ImageDataGenerator(rescale=1. / 255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.1,
        channel_shift_range=0.1)

    print('2')
    # build the VGG16 network
    model = applications.ResNet50(include_top=False, weights='imagenet')

    print('3')
    generator_train = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        #class_mode=None,
        class_mode='categorical',
        shuffle=False)
    
    print('4')
    bottleneck_features_train = model.predict_generator(
        generator_train, nb_train_samples // batch_size)
    
    print(bottleneck_features_train.shape)
    
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator_val = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        #class_mode=None,
        class_mode='categorical',
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(
        generator_val, nb_validation_samples // batch_size)
    
    print(bottleneck_features_validation.shape)
    
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    
    return bottleneck_features_train, generator_train.classes, bottleneck_features_validation,generator_val.classes 

In [32]:
def train_top_model(train_data, train_labels, 
                    validation_data,validation_labels):
    #train_data = np.load(open('bottleneck_features_train.npy','rb'))

    #train_labels = np.array(
    #    [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    #validation_data = np.load(open('bottleneck_features_validation.npy','rb'))

    train_labels = to_categorical(train_labels, num_classes=3)
    validation_labels = to_categorical(validation_labels, num_classes=3)
    
    print(train_labels.shape, train_data.shape)
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    model.summary()
    
    #model.compile(optimizer='rmsprop',
   #               loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',
            optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
            metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [19]:
train_data, train_labels, validation_data,validation_labels = save_bottlebeck_features()

1
2
3
Found 2500 images belonging to 3 classes.
4
(15928, 1, 1, 2048)
Found 560 images belonging to 3 classes.
(1600, 1, 1, 2048)


In [23]:
print(train_data.shape, train_labels.shape, validation_data.shape,validation_labels.shape)

(15928, 1, 1, 2048) (2500,) (1600, 1, 1, 2048) (560,)


In [26]:
train_data.shape[0]
train_labels_all = np.tile(train_labels, 10)

validation_data.shape[0]
validation_labels_all = np.tile(validation_labels, 10)

In [31]:
train_top_model(train_data, train_labels_all[:train_data.shape[0]], validation_data[:560],validation_labels_all[:560])

(15928, 3) (15928, 1, 1, 2048)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_13 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)               262400    
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 771       
Total params: 2,361,347
Trainable params: 2,361,347
Non-trainable params: 0
___________________________________

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #class_mode=None,
    class_mode='categorical',
    shuffle=False)

In [ ]:
imgs = np.concatenate([generator.next()[0] for i in range(generator.samples)])

In [ ]:
imgs.shape

In [ ]:
generator.classes.shape

In [ ]:
2500//16

In [ ]:
156*16

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(1, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.load_weights(top_model_weights_path)

In [ ]:
from keras.preprocessing import image

img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\41_10_ir1_191.png'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

In [ ]:
import datetime

In [ ]:
a = datetime.datetime.now()
model_vgg = applications.VGG16(include_top=False, weights='imagenet')
x_vgg_features = model_vgg.predict(x)
result = model.predict(x_vgg_features)
b = datetime.datetime.now()
print(b-a, result)